In [1]:
from code_analyzer_agent import CodeAnalyzerAgent, create_code_analysis_tools

# Configuration
CONTEXT_START = "context/v1/start-here.md"
APP_NAME = "cbv"
PATTERN = "ucc"
SOURCE_CODE = "source-code"

# Create agent with session-based logging
agent = CodeAnalyzerAgent(
    context_start_path=CONTEXT_START,
    app_name=APP_NAME,
    pattern=PATTERN,
    source_code_path=SOURCE_CODE,
    base_url="http://localhost:1234/v1",
    model="qwen/qwen3-coder-30b",
    log_dir="logs"  # Creates logs/session_TIMESTAMP/tool/ and logs/session_TIMESTAMP/llm/
)

# Register tools
tools, schemas = create_code_analysis_tools(
    source_code_path=SOURCE_CODE,
    context_dir=agent.context_dir
)

for tool_name, tool_func, schema in zip(tools.keys(), tools.values(), schemas):
    agent.register_tool(tool_name, tool_func, schema)

print("\n✅ Agent ready! Available tools:")
for tool_name in tools.keys():
    print(f"  - {tool_name}")

📁 Session logging enabled: /Users/Khaled.Alabsi/projects/llm-agent/logs/session_20251111_184658
   Tool logs: logs/session_20251111_184658/tool
   LLM logs: logs/session_20251111_184658/llm

🔍 Code Analyzer Agent Initialized
   App Name: cbv
   Pattern: ucc
   Source Code: source-code
   Context: context/v1/start-here.md
✓ Registered tool: read_file
✓ Registered tool: search_files
✓ Registered tool: grep_code
✓ Registered tool: write_analysis

✅ Agent ready! Available tools:
  - read_file
  - search_files
  - grep_code
  - write_analysis


## Example 1: Quick Exploration

Ask the agent to explore the codebase structure.

## Example 1.5: Explore Context Directory

The agent can explore both source code AND context directories using the `search_in` parameter.

In [ ]:
# Reset conversation
agent.reset_conversation()

# Task that explores BOTH directories
task = """
Explore the available context documentation:

1. Search for all markdown files in the context directory using search_in="context"
2. List the architecture documentation files
3. List the skills documentation files
4. Give me an overview of what documentation is available to guide the analysis

Use the search_in parameter to search in the context directory, not the source code.
"""

response = agent.run(task, max_iterations=5)

print("\n" + "="*80)
print("FINAL RESPONSE:")
print("="*80)
print(response)

In [ ]:
# Reset conversation for clean start
agent.reset_conversation()

# Simple exploration task
task = """
given the CBV as application 
"""

response = agent.run(task, max_iterations=5)

print("\n" + "="*80)
print("FINAL RESPONSE:")
print("="*80)
print(response)

## Example 2: Read Context and Start Discovery

Ask the agent to read the context document and begin the discovery process.

In [ ]:
# Reset for new task
agent.reset_conversation()

# Task that follows the discovery process
task = f"""
You are analyzing the '{APP_NAME}' application using the {PATTERN.upper()} pattern.

Steps:
1. Read the context document at '{CONTEXT_START}' to understand the discovery process
2. Understand Phase 1, Step 1: Discover Entrypoints
3. Search for REST controllers in the source code
4. Document what you find

Be specific about file locations and what each file does.
"""

response = agent.run(task, max_iterations=10)

print("\n" + "="*80)
print("FINAL RESPONSE:")
print("="*80)
print(response)

## Example 3: Search for Specific Code Patterns

Use the agent to find specific code patterns like @Inject, @RestController, etc.

In [ ]:
# Reset for new task
agent.reset_conversation()

# Search for specific patterns
task = """
Search the CBV application for:
1. All classes annotated with @RestController
2. All @Inject annotations (to find dependencies)
3. All interface files (*Process.java interfaces)

For each finding, tell me:
- File location
- What it does (based on class/method names)
- Any important annotations or patterns you notice
"""

response = agent.run(task, max_iterations=8)

print("\n" + "="*80)
print("FINAL RESPONSE:")
print("="*80)
print(response)

## Example 4: Generate Documentation

Ask the agent to generate migration documentation following templates.

In [ ]:
# Reset for new task
agent.reset_conversation()

# Generate documentation task
task = """
Follow the UCC discovery process to generate the entrypoints documentation:

1. Read the entrypoints template at 'context/v1/templates/entrypoints-template-ucc.md'
2. Search for all REST controllers in the CBV application
3. Search for all LSG pages
4. For each entrypoint found, document:
   - URL/endpoint
   - HTTP method
   - Request/response DTOs
   - Purpose (based on method names)
5. Write the results to 'output/entrypoints.md' using the template format

Be thorough and follow the template structure.
"""

response = agent.run(task, max_iterations=15)

print("\n" + "="*80)
print("FINAL RESPONSE:")
print("="*80)
print(response)
print("\n📝 Check output/entrypoints.md for the generated documentation")

## Check Session Logs

View the logs from the current session to track what the agent did.

In [ ]:
import os
import glob

# Show session info
if agent.session_dir:
    print(f"📁 Session Directory: {agent.session_dir}")
    print(f"📊 Total logs created: {agent.log_counter}")
    print()
    
    # List tool logs
    tool_logs = sorted(glob.glob(os.path.join(agent.tool_log_dir, "*.log")))
    print(f"🔧 Tool Logs ({len(tool_logs)}):")
    for log in tool_logs:
        print(f"  - {os.path.basename(log)}")
    
    print()
    
    # List LLM logs
    llm_logs = sorted(glob.glob(os.path.join(agent.llm_log_dir, "*.json")))
    print(f"🤖 LLM Logs ({len(llm_logs)}):")
    for log in llm_logs:
        print(f"  - {os.path.basename(log)}")
    
    print()
    print("💡 Tip: Open these files to see detailed input/output for each operation")
else:
    print("⚠️  Logging is disabled")

## Custom Task

Run your own custom analysis task.

In [ ]:
# Reset for clean start
agent.reset_conversation()

# Write your custom task here
custom_task = """
Your custom task goes here!

Examples:
- "Find all database access patterns in the CBV application"
- "List all external service calls and their endpoints"
- "Analyze the process layer and document all business logic"
- "Generate a complete UCC discovery report following all phases"

Tips:
- Be specific about what you want to find
- Reference context documents when needed (context/v1/...)
- Ask the agent to write results to output/ directory
- Use max_iterations to control how long the agent runs
"""

# Uncomment and run when you have a real task
# response = agent.run(custom_task, max_iterations=10)
# print("\n" + "="*80)
# print("FINAL RESPONSE:")
# print("="*80)
# print(response)

print("✍️  Edit the custom_task variable above with your task, then uncomment and run!")

## Utilities

Helpful utilities for working with the agent.

In [ ]:
# Utility functions

def view_conversation_history():
    """View the current conversation history"""
    print(f"📝 Conversation History ({len(agent.conversation_history)} messages):\n")
    for i, msg in enumerate(agent.conversation_history, 1):
        role = msg.get('role', 'unknown')
        content = msg.get('content', '')
        
        print(f"{i}. [{role.upper()}]")
        if content:
            preview = content[:100] + "..." if len(content) > 100 else content
            print(f"   {preview}")
        
        if 'tool_calls' in msg:
            print(f"   [Tool calls: {len(msg['tool_calls'])}]")
        
        print()

def view_last_log(log_type='tool'):
    """View the most recent log file"""
    if not agent.session_dir:
        print("⚠️  Logging is disabled")
        return
    
    log_dir = agent.tool_log_dir if log_type == 'tool' else agent.llm_log_dir
    pattern = "*.log" if log_type == 'tool' else "*.json"
    
    logs = sorted(glob.glob(os.path.join(log_dir, pattern)))
    if not logs:
        print(f"No {log_type} logs found")
        return
    
    last_log = logs[-1]
    print(f"📄 {os.path.basename(last_log)}")
    print("="*80)
    
    with open(last_log, 'r') as f:
        content = f.read()
        if log_type == 'llm':
            # Pretty print JSON
            import json
            print(json.dumps(json.loads(content), indent=2))
        else:
            print(content)

def reset_agent():
    """Reset agent and show status"""
    agent.reset_conversation()
    print("✅ Agent reset and ready for new task")
    print(f"   Session: {os.path.basename(agent.session_dir)}")
    print(f"   Total logs so far: {agent.log_counter}")

# Example usage (uncomment to use):
# view_conversation_history()
# view_last_log('tool')
# view_last_log('llm')
# reset_agent()

print("🔧 Utility functions loaded:")
print("  - view_conversation_history()")
print("  - view_last_log('tool' or 'llm')")
print("  - reset_agent()")

## Setup Agent

Initialize the agent with configuration and register tools.

# Code Analyzer Agent - CLI Interface

This notebook provides an interactive interface to the Code Analyzer Agent for analyzing the CBV application.

## What This Agent Does

The agent:
- Reads context documents (architecture, skills, templates)
- Uses tools to search and analyze source code
- Follows the UCC discovery process
- Generates migration documentation

## Available Tools

1. **read_file** - Read files from source code or context
2. **search_files** - Find files using glob patterns
3. **grep_code** - Search for code patterns
4. **write_analysis** - Write results to output files